In [1]:
#pragma cling add_include_path("./filament/include")

#pragma cling add_library_path("./build/shared_lib")
#pragma cling load("libfilament")

In [2]:
#include <filament/Engine.h>

In [3]:
using namespace filament;
using namespace utils;

In [6]:
int main() {
    Engine* engine = Engine::create(filament::Engine::Backend::OPENGL);
    // SwapChain* swapChain = engine->createSwapChain(nativeWindow);
    Renderer* renderer = engine->createRenderer();
    
    Camera* camera = engine->createCamera(EntityManager::get().create());
    View* view = engine->createView();
    Scene* scene = engine->createScene();
}

input_line_12:6:43: error: use of undeclared identifier 'EntityManager'; did you mean 'utils::EntityManager'?
    Camera* camera = engine->createCamera(EntityManager::get().create());
                                          ^~~~~~~~~~~~~
                                          utils::EntityManager
./filament/include/filament/Engine.h:28:7: note: 'utils::EntityManager' declared here
class EntityManager;
      ^
input_line_12:6:43: error: incomplete type 'utils::EntityManager' named in nested name specifier
    Camera* camera = engine->createCamera(EntityManager::get().create());
                                          ^~~~~~~~~~~~~~~
./filament/include/filament/Engine.h:28:7: note: forward declaration of 'utils::EntityManager'
class EntityManager;
      ^


Interpreter Error: 